v012 : 테스트 데이터 생성 & 로지스틱 회귀

### I'v made test data and result of logistic regression of this dataset. I'll use this data to check result of further development.

In [418]:
import pandas as pd

class Node: 
    def __init__(self): 
        self.data = None # Variable to store 'list'
        self.length = None # Length of the list
        self.next = None # Next node
        self.prev = None # Previous Node
        self.current_index = None # Index of current value of my list.

    def __str__(self): 
        return str(self.data)
    
    def get_cur_value(self):
        return self.data[self.current_index] # Returns current value of my list.

    def increse_index(self):
        overflow_flag = False # flag to show if current_index is indicating last value of my list.

        if((self.current_index + 1) == self.length): # Check if current_index is indicating the last value of my node.
            self.current_index = 0 # Resetting the current_index
            overflow_flag = True
            if(self.prev is not None):
                self.prev.increse_index() # Propagating the overflow to previous node.
        else:
            self.current_index += 1
        
        return overflow_flag
            
    
class Linked_list:
    
    def __init__(self):
        self.head_node = None
        self.tail_node = None
        self.num_node = 0
    
    def add_first(self, data):
        #create a new node
        new_node = Node()
        new_node.data = data
        new_node.length = len(data)
        new_node.current_index = 0
        
        if(self.head_node):
            new_node.next = self.head_node
            self.head_node.prev = new_node
        else:
            self.tail_node = new_node
            
        self.head_node = new_node
        Linked_list.num_node += 1
    
    def add_last(self, data):
        #create a new node
        new_node = Node()
        new_node.data = data
        new_node.length = len(data)
        new_node.current_index = 0
        
        if(self.head_node):
            new_node.prev = self.tail_node
            self.tail_node.next = new_node
        else:
            self.head_node = new_node
            
        self.tail_node = new_node
        self.num_node += 1
        
    def list_print(self):
        node = self.head_node
        while node:
            print(node.data)
            node = node.next

    def get_current_record(self):
        node = self.head_node
        return_list = []
        while node:
            return_list = return_list + [node.get_cur_value()]
            node = node.next
        return return_list
    
    def get_number_of_total_combination(self):
        total = 1
        node = self.head_node
        while(node):
            total *= node.length
            node = node.next
        return total
    
    def get_df_one(self, times=1):
        stri = [str(i) for i in list(range(1, self.num_node))]
        col_names = ["y"] + ["x" + s for s in stri]

        df = pd.DataFrame(columns = col_names)
        tot = self.get_number_of_total_combination() * times

        for i in range(tot):
            record = self.get_current_record()
            df = df.append(pd.Series(record, index = col_names), ignore_index=True)
            self.tail_node.increse_index()
        return df


In [419]:
l1 = Linked_list()
l1.add_last([1, 0])
l1.add_last(['a1', 'a2', 'a3'])
l1.add_last(['b1', 'b2', 'b3'])

In [420]:
l2 = Linked_list()
l2.add_last([0])
l2.add_last(['a1', 'a2'])
l2.add_last(['b1', 'b2'])

In [421]:
l3 = Linked_list()
l3.add_last([1])
l3.add_last(['a2', 'a3'])
l3.add_last(['b3'])

In [422]:
df1 = l1.get_df_one(1)
df2 = l2.get_df_one(10)
df3 = l3.get_df_one(20)

df = df1.append(df2).append(df3)
df = df.reset_index(drop=True)

df.to_csv('./mycsv.csv')

In [423]:
dummy_x1 = pd.get_dummies(df['x1'], prefix='x1')
dummy_x2 = pd.get_dummies(df['x2'], prefix='x2')

data = df[['y']].join(dummy_x1.ix[:,'x1_a2':]).join(dummy_x2.ix[:,'x2_b2':])
data['intercept'] = 1.0
data = data.reset_index(drop=True)

In [424]:
import statsmodels.api as sm

logit = sm.Logit(data['y'], data.ix[:,'x1_a2':])
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.280984
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   98
Model:                          Logit   Df Residuals:                       93
Method:                           MLE   Df Model:                            4
Date:                Mon, 14 Dec 2015   Pseudo R-squ.:                  0.5946
Time:                        22:10:02   Log-Likelihood:                -27.536
converged:                       True   LL-Null:                       -67.928
                                        LLR p-value:                 1.188e-16
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1_a2          0.8659      0.927      0.934      0.350        -0.951     2.683
x1_a3          2.3049      1.

----------------------------------------------------------
